In [1]:
import os
import torch as tc
import numpy as np
import open3d as otd
import pandas as pd
from tqdm import tqdm as pgb
from src.star.star import STAR
from torch.autograd import Variable
from pytorch3d.loss import point_mesh_face_distance
from pytorch3d.structures import Meshes, Pointclouds

device = tc.device("cuda" if tc.cuda.is_available() else "cpu")

## BODIES LOADING

In [2]:
all_vertices = {
    "male": [],
    "female": []
}

all_faces = {
    "male" : [],
    "female": []
}

for gender in ['female', 'male']:
    files_path = f"./data/MOVE4D/{gender}/"
    files = os.listdir(files_path)
    files.sort()
    for file in pgb(files, desc=f"loading {gender} body"):

        mesh = otd.io.read_triangle_mesh(files_path+file)
        mesh.translate([0,0,0], relative=False)
        mesh.scale(0.001, center=mesh.get_center())
        all_vertices[gender].append(tc.FloatTensor(np.array(mesh.vertices)).to(device))
        all_faces[gender].append(tc.LongTensor(np.array(mesh.triangles)).to(device))


loading male body: 100%|██████████| 72/72 [00:16<00:00,  4.38it/s]


## FUNCTIONS DEFINITIONS

In [3]:
def zero_poses(poses):
    poses = poses.clone().detach()
    poses[:,12:17] = 0
    poses[:,54:65] = 0
    poses = Variable(poses, requires_grad=True)
    return poses

def calculate_loss(vertices, faces, scan_meshes, scan_pc):
    star_meshes = Meshes(verts=vertices, faces=faces)
    s2m = point_mesh_face_distance(star_meshes, scan_pc)
    return s2m


## BODIES FITTING

In [4]:
def training(scan_mesh, scan_pc, steps, step_interval, gender, rep):
    
    original_steps = steps
    nbetas=0
    poses = tc.FloatTensor(tc.zeros((1, 72))).to(device)
    poses = Variable(poses, requires_grad=True)
    betas = tc.FloatTensor(tc.zeros((1, nbetas))).to(device)
    betas = Variable(betas, requires_grad=True)
    trans = tc.FloatTensor(tc.zeros((1, 3))).to(device)
    trans = Variable(trans, requires_grad=True)
    star = STAR(gender=gender, num_betas=nbetas)
    star_faces = np.repeat(star.faces[None,...],1,axis=0).to(device)

    # training model
    learning_rate = 0.02
    epochs = 10000
    epochs_iterator = pgb(range(epochs), desc=f"optimizing {gender} - loss: inf", position=2)
    for it in epochs_iterator:

        elapsed_time = epochs_iterator.format_dict['elapsed']
        
        if it == 20:
            poses = zero_poses(poses)

        if it%step_interval == 0:

            if nbetas < 300:
                nbetas += steps
                if nbetas > 300:
                    temp_steps = nbetas - 300
                    steps = steps - temp_steps
                    nbetas = 300
                new_betas = tc.FloatTensor(np.zeros((1, nbetas))).to(device)
                new_betas[:,:nbetas-steps] = betas
                betas = Variable(new_betas, requires_grad=True)
                star = STAR(gender=gender, num_betas=nbetas)

            optimizer = tc.optim.Adam([trans, betas, poses], lr=learning_rate)
        d = star(betas=betas, pose=poses, trans=trans)
        optimizer.zero_grad()
        loss = calculate_loss(d, star_faces, scan_mesh,scan_pc)
        loss.backward(retain_graph=True)
        optimizer.step()
        epochs_iterator.set_description(f"optimizing {gender} - loss: {loss}")
        if loss < 2e-5:
            break
        if elapsed_time > 300:
            break
    return [gender, it, loss.detach().cpu().numpy(), elapsed_time, original_steps, step_interval, rep]

In [5]:
try:
    status_data = pd.read_csv("status_data_4.csv", index_col=0)
except Exception as e:
    status_data = pd.DataFrame(columns=["gender", 'iteration', 'loss', 'elapsed_time', 'steps', 'steps interval', 'rep'])

for gender in ["female", 'male']:

    scan_mesh = Meshes(verts=all_vertices[gender], faces=all_faces[gender])[0]
    scan_pc = Pointclouds(points=all_vertices[gender])[0]
    repetition_iteration = pgb(range(10), desc="repetition iterator", position=0)
    for rep in repetition_iteration:
        interval_iterator = pgb([200,250,300], desc="interval iterator", position=1)
        for interval in interval_iterator:
            steps_iterator = pgb([250], desc="steps iterator", position=2)
            for steps in steps_iterator:
                status_data.loc[len(status_data.index)] = training(scan_mesh, scan_pc, steps, interval, gender, rep)
                status_data.to_csv("status_data_4.csv")


repetition iterator:   0%|          | 0/10 [00:00<?, ?it/s]












































































































































































































































































































































































































































































































































































































































































































































































































































































































































































